# Programación rápida  con Julia (en serie)

Sí, este es un curso de computación paralela, pero para escribir programas paralelos eficientes, primero debemos aprender a escribir código serial Julia rápido. Esta es una introducción rápida a la programación de alto rendimiento (en serie).

_Recomiendo mucho_ revisar los Consejos de rendimiento en el manual [Performance Tips](https://docs.julialang.org/en/v1.1/manual/performance-tips/). Esto sólo va a introducir brevemente algunos de los conceptos principales.

## Medir, medir, medir.
Es muy fácil experimentar en Julia; puede probar rápidamente muchas opciones y ver cuál es la más rápida.

Usa el paquete  [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl):

In [9]:
? findmin

search: findmin findmin! findmax findmax!



```
findmin(f, domain) -> (f(x), index)
```

Returns a pair of a value in the codomain (outputs of `f`) and the index of the corresponding value in the `domain` (inputs to `f`) such that `f(x)` is minimised. If there are multiple minimal points, then the first one will be returned.

`domain` must be a non-empty iterable.

`NaN` is treated as less than all other values except `missing`.

!!! compat "Julia 1.7"
    This method requires Julia 1.7 or later.


# Examples

```jldoctest
julia> findmin(identity, 5:9)
(5, 1)

julia> findmin(-, 1:10)
(-10, 10)

julia> findmin(first, [(2, :a), (2, :b), (3, :c)])
(2, 1)

julia> findmin(cos, 0:π/2:2π)
(-1.0, 3)
```

---

```
findmin(itr) -> (x, index)
```

Return the minimal element of the collection `itr` and its index or key. If there are multiple minimal elements, then the first one will be returned. `NaN` is treated as less than all other values except `missing`.

See also: [`findmax`](@ref), [`argmin`](@ref), [`minimum`](@ref).

# Examples

```jldoctest
julia> findmin([8, 0.1, -9, pi])
(-9.0, 3)

julia> findmin([1, 7, 7, 6])
(1, 1)

julia> findmin([1, 7, 7, NaN])
(NaN, 4)
```

---

```
findmin(A; dims) -> (minval, index)
```

For an array input, returns the value and index of the minimum over the given dimensions. `NaN` is treated as less than all other values except `missing`.

# Examples

```jldoctest
julia> A = [1.0 2; 3 4]
2×2 Matrix{Float64}:
 1.0  2.0
 3.0  4.0

julia> findmin(A, dims=1)
([1.0 2.0], CartesianIndex{2}[CartesianIndex(1, 1) CartesianIndex(1, 2)])

julia> findmin(A, dims=2)
([1.0; 3.0;;], CartesianIndex{2}[CartesianIndex(1, 1); CartesianIndex(2, 1);;])
```


In [2]:
using BenchmarkTools

"""
    findclosest(data, point)

Un ejemplo simple que regresa el elemento en `data`que esta mas cercano al punto dado `point`.
"""
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
data = rand(5000)
findclosest(data, 0.5)

0.5001596495221731

In [11]:
#data = rand(5000)
findmin(abs.(data .- 0.5))

(0.0001596495221730665, 375)

In [4]:
@btime findclosest($data, $0.5)

  13.658 μs (2 allocations: 39.11 KiB)


0.5001596495221731

In [5]:
@benchmark findclosest($data, $0.5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  14.021 μs … 758.309 μs  ┊ GC (min … max): 0.00% … 96.67%
 Time  (median):     14.647 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.273 μs ±  20.557 μs  ┊ GC (mean ± σ):  3.50% ±  2.74%

  ▃▇█▇▅▂                             ▁▃▂  ▃▃▂ ▁                ▂
  ███████▇▇▄▆▄▄▁▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▆▅██████████▇▆▃▁▄▃▆▆▅▄▆▇▆▅▅ █
  14 μs         Histogram: log(frequency) by time      27.6 μs <

 Memory estimate: 39.11 KiB, allocs estimate: 2.

### Profile!

In [6]:
using Profile

Profile.clear()
@profile for _ in 1:1000; findclosest(data, 0.5); end

Profile.print(maxdepth=11)

Overhead ╎ [+additional indent] Count File:Line; Function
  ╎46 @Base/task.jl:484; (::IJulia.var"#15#18")()
  ╎ 46 @IJulia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
  ╎  46 @Base/essentials.jl:726; invokelatest
  ╎   46 @Base/essentials.jl:729; #invokelatest#2
  ╎    46 ...c/execute_request.jl:67; execute_request(socket::ZMQ.Soc...
  ╎     46 .../SoftGlobalScope.jl:65; softscope_include_string(m::Mod...
  ╎    ╎ 46 @Base/loading.jl:1428; include_string(mapexpr::typeo...
  ╎    ╎  46 @Base/boot.jl:368; eval
  ╎    ╎   46 ...ile/src/Profile.jl:27; top-level scope
 1╎    ╎    46 In[6]:4; macro expansion
  ╎    ╎     45 In[2]:9; findclosest(data::Vector{Flo...
  ╎    ╎    ╎ 23 @Base/broadcast.jl:860; materialize
  ╎    ╎    ╎ 22 @Base/reducedim.jl:1112; findmin
Total snapshots: 46. Utilization: 100% across all threads and tasks. Use the `groupby` kwarg to break down by thread and/or task


### Iterar!

Antes teniamos:
```julia
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
```

Propongamos una nueva definición que pueda combinar las dos operaciones:

In [12]:
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end

# And do a spot-check to make sure we did the optimization correctly:
findclosest2(data, 0.5) == findclosest(data, 0.5)

true

In [13]:
@benchmark findclosest2($data, $0.5)

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.730 μs …  10.857 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.768 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.807 μs ± 303.300 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▆▂  ▃▂                                                    ▁
  ████▁▃██▇▅▅▇▆▅▅▄▄▁▄▃▁▁▃▅▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▄▃▅▅▆▅ █
  4.73 μs      Histogram: log(frequency) by time      6.09 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
@btime findclosest2($data, $0.5)

  4.732 μs (0 allocations: 0 bytes)


0.5001596495221731

## Una palabra rápida en las macros

Las macros son esas cosas divertidas que comienzan con `@`. Pueden reinterpretar lo que
escribes y hacer algo diferente, esencialmente introduciendo una nueva palabra clave.

Por ejemplo, la macro `@assert` simplemente toma una expresión y lanza un
excepción si devuelve `falso`.

In [17]:
@assert 2+2 == 4

In [19]:
@assert 2+2 == 8

LoadError: AssertionError: 2 + 2 == 8

Lo hace literalmente reescribiendo lo que escribiste. Puedes verlo en acción
con `@macroexpand`

In [18]:
@macroexpand @assert 2+2 == 4

:(if 2 + 2 == 4
      nothing
  else
      Base.throw(Base.AssertionError("2 + 2 == 4"))
  end)

In [20]:
@macroexpand @time 2+2 == 4

quote
    #= timing.jl:252 =#
    begin
        #= timing.jl:257 =#
        $(Expr(:meta, :force_compile))
        #= timing.jl:258 =#
        local var"#33#stats" = Base.gc_num()
        #= timing.jl:259 =#
        local var"#35#elapsedtime" = Base.time_ns()
        #= timing.jl:260 =#
        Base.cumulative_compile_timing(true)
        #= timing.jl:261 =#
        local var"#36#compile_elapsedtimes" = Base.cumulative_compile_time_ns()
        #= timing.jl:262 =#
        local var"#34#val" = $(Expr(:tryfinally, :(2 + 2 == 4), quote
    var"#35#elapsedtime" = Base.time_ns() - var"#35#elapsedtime"
    #= timing.jl:264 =#
    Base.cumulative_compile_timing(false)
    #= timing.jl:265 =#
    var"#36#compile_elapsedtimes" = Base.cumulative_compile_time_ns() .- var"#36#compile_elapsedtimes"
end))
        #= timing.jl:267 =#
        local var"#37#diff" = Base.GC_Diff(Base.gc_num(), var"#33#stats")
        #= timing.jl:268 =#
        local var"#38#_msg" = Base.nothing
        #= timing.jl:269

In [21]:
@which @time 2+2 == 4

var"@time"(__source__::LineNumberNode, __module__::Module, ex) in Base at timing.jl:250

Cada macro puede definir su propia sintaxis especial, y esto se usa ampliamente para la introspección de código, las mejoras de rendimiento en serie y, quizás lo más importante, ¡pprivilegios de paralelización!

## ¿Qué tan rápido es Julia?

Al comprender los conceptos básicos de cómo Julia _puede_ ser rápida, puede obtener una mejor
sentido de cómo escribir código Julia rápido.

Perhaps most importantly, Julia can reason about types. Recall: this is the definition of `findclosest2`:

```julia
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end
```

In [22]:
@code_typed optimize=false findclosest2(data, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Float64
│   %2  = (bestval - point)::Float64
│         (bestdist = Main.abs(%2))::Float64
│   %4  = data::Vector{Float64}
│         (@_4 = Base.iterate(%4))::Union{Nothing, Tuple{Float64, Int64}}
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Float64, Int64}
│         (elt = Core.getfield(%9, 1))::Float64
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Float64
│         (dist = Main.abs(%12))::Float64
│   %14 = (dist < bestdist)::Bool
└──       goto #4 if not %14
3 ─       (bestval = elt)::Float64
└──       (bestdist = dist)::Float64
4 ┄       (@_4 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Float64, Int64}}
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Float64

In [23]:
typeof(data)

Vector{Float64} (alias for Array{Float64, 1})

In [24]:
newdata = Real[data...]
typeof(newdata)

Vector{Real} (alias for Array{Real, 1})

In [25]:
@code_typed optimize=false findclosest2(newdata, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Real
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))::Any
│   %4  = data::Vector{Real}
│         (@_4 = Base.iterate(%4))::Union{Nothing, Tuple{Real, Int64}}
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Real, Int64}
│         (elt = Core.getfield(%9, 1))::Real
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))::Any
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)::Real
└──       (bestdist = dist)::Any
4 ┄       (@_4 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Real, Int64}}
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Real

In [26]:
@benchmark findclosest2($newdata, $0.5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  146.229 μs …  1.984 ms  ┊ GC (min … max): 0.00% … 90.97%
 Time  (median):     153.479 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   163.144 μs ± 78.498 μs  ┊ GC (mean ± σ):  2.73% ±  5.24%

  ▃ █▃▃             ▁▁                                         ▁
  █████▇█▇▇▇▇▇▇▇▆▆▆▇██▇▆▆▅▅▄▄▅▄▄▄▅▄▃▃▃▃▂▂▂▃▂▄▃▂▃▃▃▃▂▃▂▃▄▃▄▄▄▅▆ █
  146 μs        Histogram: log(frequency) by time       310 μs <

 Memory estimate: 156.28 KiB, allocs estimate: 10002.

In [27]:
@code_warntype findclosest2(newdata, 0.5)

MethodInstance for findclosest2(::Vector{Real}, ::Float64)
  from findclosest2(data, point) in Main at In[12]:1
Arguments
  #self#::Core.Const(findclosest2)
  data::Vector{Real}
  point::Float64
Locals
  @_4::Union{Nothing, Tuple{Real, Int64}}
  bestdist::Any
  bestval::Real
  elt::Real
  dist::Any
Body::Real
1 ─       (bestval = Main.first(data))
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))
│   %4  = data::Vector{Real}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Real, Int64}
│         (elt = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)
└──       (bestdist = dist)
4 ┄       (@_4 = Base.iterate(%4, %11))
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not 

In [28]:
@code_warntype findclosest2(data, 0.5)

MethodInstance for findclosest2(::Vector{Float64}, ::Float64)
  from findclosest2(data, point) in Main at In[12]:1
Arguments
  #self#::Core.Const(findclosest2)
  data::Vector{Float64}
  point::Float64
Locals
  @_4::Union{Nothing, Tuple{Float64, Int64}}
  bestdist::Float64
  bestval::Float64
  elt::Float64
  dist::Float64
Body::Float64
1 ─       (bestval = Main.first(data))
│   %2  = (bestval - point)::Float64
│         (bestdist = Main.abs(%2))
│   %4  = data::Vector{Float64}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Float64, Int64}
│         (elt = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Float64
│         (dist = Main.abs(%12))
│   %14 = (dist < bestdist)::Bool
└──       goto #4 if not %14
3 ─       (bestval = elt)
└──       (bestdist = dist)
4 ┄       (@_4 = Base.iterate(%4, %11))
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not

### Tipo de estabilidad

Una función se denomina de tipo estable si Julia puede inferir cuál será el tipo de salida basándose únicamente en los tipos de las entradas.

Cosas que frustran la estabilidad del tipo:

* Ejecutar cosas en el ámbito global: ¡cree funciones en su lugar!
* Contenedores de un tipo no concreto
* Estructuras con campos de tipo abstracto
* Globales no constantes (¡podrían cambiar!)
* Funciones que cambian lo que devuelven en función de los _valores_ :

#### Más sobre macros
Todas y cada una de las macros pueden definir su propia sintaxis. La macro @benchmark usa $ de una manera especial. El objetivo detrás de @benchmark es evaluar el rendimiento de un fragmento de código como si estuviera escrito en una función. Use $ para marcar lo que será un argumento o una variable local en la función. Olvidarse de usar $ puede resultar en tiempos más rápidos o más lentos que el rendimiento del mundo real.

In [29]:
x = 0.5 # non-constant global
@btime sin(x)
@btime sin($x)

  18.578 ns (1 allocation: 16 bytes)
  5.284 ns (0 allocations: 0 bytes)


0.479425538604203

In [30]:
@btime sin(0.5) # constant literal!
@btime sin($0.5)

  1.268 ns (0 allocations: 0 bytes)
  5.284 ns (0 allocations: 0 bytes)


0.479425538604203

In [31]:
x=1
f()=sin(x)

f (generic function with 1 method)

In [33]:
@btime f()

  22.847 ns (1 allocation: 16 bytes)


0.8414709848078965

In [34]:
g(x)=sin(x)

g (generic function with 1 method)

In [35]:
@btime g(10.5)

  1.269 ns (0 allocations: 0 bytes)


-0.87969575997167

In [36]:
t=10.5
@btime g($t)

  9.798 ns (0 allocations: 0 bytes)


-0.87969575997167